<a href="https://colab.research.google.com/github/klajosw/python/blob/master/kl_py_img_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI képek átalakítása tensorflow-val

## Ez az oktatóprogram mélyreható tanuláson alapszik.

Egy képet egy másik kép stílusában komponál át.

Játékosan elérhető hogy hasonló kép készüljön mint Picasso vagy Van Gogh alkotása. 

### Ezt neurális stílusátvitelnek nevezük, és a technikát a művészi stílus neurális algoritmusával valósítjük meg.

Megjegyzés: 
 - Ez az oktatóanyag bemutatja az eredeti stílusátviteli algoritmust. 
 
 - Optimalizálja a kép tartalmát egy adott stílushoz. 
 
 - A modern és antik megközelítések egy-egy modellt képezhet a stilizált kép közvetlen generálására (hasonlóan a cycganhez). 
 
 Ez a megközelítés sokkal gyorsabb (akár 1000x). Egy előre kiképzett tetszőleges képstilizáló modul elérhető a TensorFlow Hub-ban és a TensorFlow Lite-hoz.

A neurális stílusátvitel egy olyan optimalizálási technika, amelyet két kép - egy tartalomkép és egy stílus referenciakép (például egy híres festőművész alkotása) készítéséhez használnak, és összekeverik őket, így a kimeneti kép úgy néz ki, mint a tartalomkép, de “festett” a stílus referenciaképének stílusában.

Ezt úgy valósítják meg, hogy a kimeneti képet úgy optimalizálják, hogy megfeleljen a tartalom kép tartalmi statisztikájának és a stílus referencia kép stílus statisztikájának. Ezeket a statisztikákat a képekből egy konvolúciós hálózat segítségével vonják ki.



### Minta forrása:
https://www.tensorflow.org/tutorials/generative/style_transfer



Kiinduló kép egy mokus a farönkön

<img src="https://1.bp.blogspot.com/-1xfgXDl6mYQ/XmIwnys8MDI/AAAAAAAAaFw/16mXksGvTiE3kgdXzU4LTNkM9YVnbvHXwCLcBGAsYHQ/s320/FB_IMG_1580926207371.jpg" width="500px"/>


Stilus kép:

[stilus](https://1.bp.blogspot.com/-PA6HO5WxmMo/XmIxUVZsLbI/AAAAAAAAaGI/n4jHuu8NctsdnoFDzvy0mSHAuRJKdYlFwCLcBGAsYHQ/s320/1-style.jpg), 

<img src="https://1.bp.blogspot.com/-PA6HO5WxmMo/XmIxUVZsLbI/AAAAAAAAaGI/n4jHuu8NctsdnoFDzvy0mSHAuRJKdYlFwCLcBGAsYHQ/s320/1-style.jpg" width="500px"/>


Eredmény kép:

<img src="https://1.bp.blogspot.com/-GE6DF_B8Kaw/XmJBYvAAJHI/AAAAAAAAaGU/Z_t7Mqp66Ps1rgHnrK1SGzfQWTxwgwxmgCLcBGAsYHQ/s320/mokus_stilus2.png" style="width: 500px;"/>

---

## Vizualizácios bemenetek

Itt határozzuk meg a kép betöltésének funkcióját, és korlátozzuk annak maximális méretét 512 pixelre.

### Több képkezelő eljárás kerül meghatározásra

---

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
## tensorflow és más modulok betöltése
try:
  # %tensorflow_version ellenőrzése
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

## Veziok kiírása
print('TF verzio    : ', tf.__version__)
print('KERAS verzio : ', tf.keras.__version__)

In [0]:
## képállományok elérése a WEB -en
content_path = tf.keras.utils.get_file('forras_kep.jpg', r'https://1.bp.blogspot.com/-1xfgXDl6mYQ/XmIwnys8MDI/AAAAAAAAaFw/16mXksGvTiE3kgdXzU4LTNkM9YVnbvHXwCLcBGAsYHQ/s320/FB_IMG_1580926207371.jpg')
style_path   = tf.keras.utils.get_file('cel_kep.jpg', r'https://1.bp.blogspot.com/-PA6HO5WxmMo/XmIxUVZsLbI/AAAAAAAAaGI/n4jHuu8NctsdnoFDzvy0mSHAuRJKdYlFwCLcBGAsYHQ/s320/1-style.jpg')


In [0]:
## kép brtöltő eljárás definiciója
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

def load_img(path_to_img):
  max_dim = 512  ## max
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

## kép megjelenítés
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [0]:
## eljárások meghívása
content_image = load_img(content_path) ## képfile bolvasása
style_image = load_img(style_path)     ## stilus képfile bolvasása

plt.subplot(1, 2, 1)                   ## megjelenítési pozició
imshow(content_image, 'Eredeti kép') ## megjelenítés

plt.subplot(1, 2, 2)                   ## megjelenítési pozició
imshow(style_image, 'Stilus kép')     ## megjelenítés

---

##Gyors stílusátvitel a TF-Hub segítségével lehet megtámogatni

### Elérése:

 [TensorFlow Hub](https://tensorflow.org/hub) 

 
 ---

In [0]:
## A hub használathoz installálni kell a modult (ezt csak egyszer kell megtenni)
# !pip install "tensorflow_hub>=0.6.0"

In [0]:
import tensorflow_hub as hub
#hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')  ## regi verzio
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')  ## újjabb verzio
stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)

## A tartalom és a stílus ábrázolás paramétereinek megadása

Használja a modell köztes rétegeit a kép tartalmának és stílusának ábrázolásához. 

A hálózat bemeneti rétegétől kezdve az első néhány réteg aktiválás alacsony szintű funkciókat képvisel, például éleket és textúrákat.

 A hálózaton való áthaladás során az utolsó néhány réteg magasabb szintű funkciókat képvisel - objektumrészeket, mint például a kerekeket vagy a szemét. 

Ebben az esetben a VGG19 hálózati architektúrát használjujk egy előre képzett képosztályozó hálózatot. 

Ezekre a köztes rétegekre van szükség a tartalom és a stílus képek ábrázolásának meghatározásához. 

A bemeneti képhez próbáljuk meg egyeztetni a megfelelő stílus- és tartalomcél reprezentációkat ezeken a közbenső rétegeken.

Model elérése:
 [VGG19](https://keras.io/applications/#vgg19) 


In [0]:
## model betoltés és paraméterezés
x = tf.keras.applications.vgg19.preprocess_input(content_image*255)
x = tf.image.resize(x, (224, 224))
vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
prediction_probabilities = vgg(x)
prediction_probabilities.shape

In [0]:
## Prediktiv modell
predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
[(class_name, prob) for (number, class_name, prob) in predicted_top_5]

In [0]:
##  VGG19-et betöltése az osztályozó fej nélkül, és  rétegnevek felsorolása nélkül
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

print()
for layer in vgg.layers:
  print(layer.name)

Choose intermediate layers from the network to represent the style and content of the image:


In [0]:
## Válasszunk köztes rétegek a hálózatból, hogy hogyan ábrázolja a kép stílusát és tartalmát:
## Tartalmi réteg, ahova behúzza a funkciótérképeket
content_layers = ['block5_conv2'] 

# Stilus réteg megadása
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

## Középső rétegek a stílus és a tartalom számára

Ezek a közbenső kimenetek engedik meg az előre képzett képosztályozó hálózatunkban a stílus és a tartalom ábrázolásainak meghatározását.

Magas szinten annak érdekében, hogy egy hálózat elvégezhesse a kép osztályozását (amelyet erre a hálózatra kiképeztek), meg kell értenie a képet.

 Ehhez a nyers képet bemeneti képpontként kell venni, és létre kell hozni egy olyan belső ábrázolást, amely a nyers kép pixeleket a képen lévő tulajdonságok komplex megértésévé alakítja.

Ez is az oka annak, hogy a konvolúciós ideghálózatok képesek jól általánosítani: képesek megragadni az osztályokon belüli invarianciákat és meghatározó jellemzőket (pl. Macskák vagy kutyák), amelyek a háttérzajt és más kellemetlenségeket kiváltják. 

Így, valahol a nyers kép beillesztése a modellbe és a kimeneti osztályozási címke között, a modell komplex szolgáltatáskivonatként szolgál. A modell köztes rétegeinek elérésével leírhatja a bemeneti képek tartalmát és stílusát.

## Ezek után készítsuk el a modellt

A tf.keras.applications hálózatait úgy tervezték meg, hogy a Keras funkcionális API segítségével könnyedén kivonhassák a közbenső réteg értékeit.

A modell meghatározásához a funkcionális API használatával adja meg a bemeneteket és a kimeneteket:

Ez a következő függvény egy VGG19 modellt épít fel, amely visszaadja a közbenső réteg kimeneteinek listáját:

In [0]:
def vgg_layers(layer_names):
  # # Töltsünk be modellünket. Töltsük be az előképzett VGG-t, a képzett az imagenet adatokon
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model

In [0]:
style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style_image*255)

##  Nézzük meg az egyes rétegek kimeneti statisztikáit
for name, output in zip(style_layers, style_outputs):
  print(name)
  print("  shape: ", output.numpy().shape)
  print("  min: ", output.numpy().min())
  print("  max: ", output.numpy().max())
  print("  mean: ", output.numpy().mean())
  print()

---

## Stilus számítások

A kép tartalmát a közbenső jellemző térképek értékei képviselik.

Kiderült, hogy a kép stílusát leírhatjuk az eszközökkel és a különféle térképek közötti összefüggésekkel. 

Számítson ki egy Gram-mátrixot, amely tartalmazza ezt az információt, úgy, hogy az objektumvektor külső termékét magával veszi az egyes helyekre, és átlagolja ezt a külső terméket az összes helyre. 


Ez a Gram-mátrix egy adott rétegre kiszámítható:

$$G^l_{cd} = \frac{\sum_{ij} F^l_{ijc}(x)F^l_{ijd}(x)}{IJ}$$

Ez tömören megvalósítható a `tf.linalg.einsum` függvény használatával: 

---

In [0]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

---

## Kivonatolt stílus és tartalom

Készítsünk egy modellt, amely visszatér a stílus és a tartalom informáciokkal tenzoraihoz.

---

In [0]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}

---

Amikor képre hívnak egy képet, ez a modell visszatér a "stílus_rétegek" grammátrixa (stílusa) és a "tartalom_rétegek" tartalmával.

---

In [0]:
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(content_image))

style_results = results['style']

print('Styles:')
for name, output in sorted(results['style'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())
  print()

print("Contents:")
for name, output in sorted(results['content'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())


In [0]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

image = tf.Variable(content_image) ##  tartalmazzák a képet az optimalizálás érdekében. A gyors megvalósítás érdekében inicializálja a tartalomképpel

def clip_0_1(image):  ## Mivel ez úszó kép, definiáljon egy funkciót a pixelértékek 0 és 1 közötti tartására:
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)  ## optimalizáslás beállításai

style_weight=1e-2
content_weight=1e4

In [0]:
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

In [0]:
@tf.function()
def train_step(image):  ## kép változtatási lépések
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

---

### futtassünk pár válzoztató lépés közvetlenül és nézzük meg a hatását

---

In [0]:
train_step(image)      ## 1 lépés
train_step(image)      ## 2 lépés
train_step(image)      ## 3 lépés
train_step(image)      ## 4 lépés
train_step(image)      ## 5 lépés
tensor_to_image(image) # megjelenítés

---

## Mivel eddig működik, végezzünk optimalizálást is

---


In [0]:
import time
start = time.time()

epochs = 5            ## léptető paraméter 
steps_per_epoch = 80  ## léptető paraméter 

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))
  
end = time.time()
print("Total time: {:.1f}".format(end-start))

---

## Teljes variációs veszteség

Ennek az alapvető megvalósításnak az egyik hátránya, hogy sok magas frekvenciájú művet hoz létre. 

Csökkentse ezeket a kép nagyfrekvenciájú összetevőinek kifejezett szabályzási kifejezés használatával. 

A stílusátvitel során ezt gyakran így hívják: *total variation loss*


---

In [0]:
def high_pass_x_y(image):
  x_var = image[:,:,1:,:] - image[:,:,:-1,:]
  y_var = image[:,1:,:,:] - image[:,:-1,:,:]

  return x_var, y_var

In [0]:
x_deltas, y_deltas = high_pass_x_y(content_image)

plt.figure(figsize=(14,10))
plt.subplot(2,2,1)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Original")

plt.subplot(2,2,2)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Original")

x_deltas, y_deltas = high_pass_x_y(image)

plt.subplot(2,2,3)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Styled")

plt.subplot(2,2,4)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Styled")

---

## Most megmutatjuk, hogyan nővelhető a magas frekvenciájú komponensek.

Ez a magas frekvenciájú elem alapvetõen egy éledetektor is. 

Hasonló kimenetet kaphatunk például a Sobel éldetektorról, például:

---

In [0]:
plt.figure(figsize=(14,10))

sobel = tf.image.sobel_edges(content_image)
plt.subplot(1,2,1)
imshow(clip_0_1(sobel[...,0]/4+0.5), "Horizontal Sobel-edges")
plt.subplot(1,2,2)
imshow(clip_0_1(sobel[...,1]/4+0.5), "Vertical Sobel-edges")

The regularization loss associated with this is the sum of the squares of the values:

In [0]:
def total_variation_loss(image):
  x_deltas, y_deltas = high_pass_x_y(image)
  return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))

total_variation_loss(image).numpy()

tf.image.total_variation(image).numpy()

## Optimalizáció ujra futtatása

Válasszon súlyt  `total_variation_loss`:

In [0]:
total_variation_weight=30

@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    loss += total_variation_weight*tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

image = tf.Variable(content_image)

In [0]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))

end = time.time()
print("Total time: {:.1f}".format(end-start))

---

## végül mentsuk az eredményünket

---

In [0]:
file_name = 'stylized-mokus.png'
tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)